# This model converts python code to more performant C++ code

In [1]:
# imports

import os
import io
import sys
from dotenv import load_dotenv
from openai import OpenAI
import subprocess

import gradio as gr

In [2]:
load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')

In [3]:
openai = OpenAI()

In [4]:
def execute_python(code):
        try:
            output = io.StringIO()
            sys.stdout = output
            exec(code)
        finally:
            sys.stdout = sys.__stdout__
        return output.getvalue()

In [5]:
# write to a file called optimized.cpp

def write_output(cpp):
    code = cpp.replace("```cpp","").replace("```","")
    with open("assets/optimized.cpp", "w") as f:
        f.write(code)

In [6]:
def execute_cpp(code):
        write_output(code)
        try:
            # compile_cmd = ["clang++", "-Ofast", "-std=c++17", "-march=armv8.5-a", "-mtune=apple-m1", "-mcpu=apple-m1", "-o", "optimized", "optimized.cpp"]
            compile_cmd = ["g++", "-std=c++17", "-o", "assets/optimized", "assets/optimized.cpp"]
            compile_result = subprocess.run(compile_cmd, check=True, text=True, capture_output=True)
            run_cmd = ["./assets/optimized"]
            run_result = subprocess.run(run_cmd, check=True, text=True, capture_output=True)
            return run_result.stdout
        except subprocess.CalledProcessError as e:
            return f"An error occurred:\n{e.stderr}"

In [7]:
system_message = """You are an AI assistant specialized in converting Python code to C++ 
                    while preserving functionality and efficiency. Ensure proper syntax, memory management,
                    and performance optimization. Maintain readability and follow C++ best practices.
                    Use comments sparingly and do not provide any explanation other than occasional comments."""

In [8]:

def stream_gpt(prompt):
    messages = [
        {"role": "system", "content": system_message},
        {"role": "system", "content": prompt}
    ]

    stream = openai.chat.completions.create(
        model='gpt-4o-mini',
        messages=messages,
        stream=True
    )
    result = ""
    for chunk in stream:
        result += chunk.choices[0].delta.content or ""
        yield result

In [9]:
# Demo code to calculate value of pi 
pi = """
import time

def calculate(iterations, param1, param2):
    result = 1.0
    for i in range(1, iterations+1):
        j = i * param1 - param2
        result -= (1/j)
        j = i * param1 + param2
        result += (1/j)
    return result

start_time = time.time()
result = calculate(100_000_000, 4, 1) * 4
end_time = time.time()

print(f"Result: {result:.12f}")
print(f"Execution Time: {(end_time - start_time):.6f} seconds")
"""

In [10]:
with gr.Blocks() as ui:
    gr.Markdown(" ## Convert Python code to C++")
    with gr.Row():
        python = gr.Textbox(label="Enter python code", value=pi, lines=10)
        cpp = gr.Textbox(label="C++ code", lines=10)
    with gr.Row():
        convert = gr.Button("Convert code")
    with gr.Row():
        run_python = gr.Button("Execute Python")
        run_cpp = gr.Button("Execute C++")
    with gr.Row():
        out_python = gr.TextArea("Python result", elem_classes=["python"])
        out_cpp = gr.TextArea("C++ result", elem_classes=["cpp"])

    convert.click(stream_gpt, inputs=[python], outputs=[cpp])
    run_python.click(execute_python, inputs=[python], outputs=[out_python])
    run_cpp.click(execute_cpp, inputs=[cpp], outputs=[out_cpp])

ui.launch()

* Running on local URL:  http://127.0.0.1:7862

To create a public link, set `share=True` in `launch()`.
